## How to use

Provide a time-domain data series, saved as a xlsx file. The format of the matrix is:

1st column: Time axis
Remaining columns: Dependant variable data

Specify an intended interval for the data. 

The program will erase all samples which are not in line with the interval.

Intended use: FEM results (Comsol) in which convergence issues populate the data with extraneous samples.

In [166]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.decomposition import PCA as RandomizedPCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
import pickle
from PIL import Image
from pathlib import Path 

In [167]:
# datafile = 'G:\My Drive'
datafile = "I:\\My Drive\\USGW Research\\COMSOL Models\\Pipe 4\\Raw Model Signals\\Viscoelastic Models\\Varying Depth\\Radial Displacements 0.5-15 mm Mesh 10 mm Thickness Inner Circumference.xlsx"

In [168]:
sheetname = '21 mm deep 30 deg'
data = pd.read_excel(datafile, sheetname) 
data = data.set_index('Time (ms)')
data = data.transpose()

data

Time (ms)                                                   0.000  \
Radial Displacement A1 (mm), Point: (21, ro, 0)     -1.874919e-49   
Radial Displacement A2 (mm), Point: (21, 0, ro)     -1.878951e-49   
Radial Displacement A3 (mm), Point: (21, -ro, 0)    -1.874838e-49   
Radial Displacement A4 (mm), Point: (21, 0, -ro)    -1.874939e-49   
Radial Displacement A1.5 (mm), Point: (21, 0.5*...  -1.847403e-49   
Radial Displacement A2.5 (mm), Point: (21, -0.5...  -1.846380e-49   
Radial Displacement A3.5 (mm), Point: (21, -0.5...  -1.849960e-49   
Radial Displacement A4.5 (mm), Point: (21, 0.5*...  -1.850419e-49   
Radial Displacement A'1 (mm), Point: (420, ro, 0)    3.665651e-78   
Radial Displacement A'2 (mm), Point: (420, 0, ro)    3.119683e-79   
Radial Displacement A'3 (mm), Point: (420, -ro, 0)   3.800742e-78   
Radial Displacement A'4 (mm), Point: (420, 0, -ro)   3.273797e-78   
Radial Displacement A'1.5 (mm), Point: (420, 0....   1.534367e-79   
Radial Displacement A'2.5 (mm), Point: (420, -0...  -1.346842e-79   
Radial Displacement A'3.5 (mm), Point: (420, -0...  -2.857131e-79   
Radial Displacement A'4.5 (mm), Point: (420, 0....   3.872548e-79   
Radial Displacement R'1 (mm), Point: (792, ro, 0)   2.624637e-103   
Radial Displacement R'2 (mm), Point: (792, 0, ro)   2.471391e-104   
Radial Displacement R'3 (mm), Point: (792, -ro, 0)  2.763374e-103   
Radial Displacement R'4 (mm), Point: (792, 0, -ro)  4.284595e-103   
Radial Displacement R'1.5 (mm), Point: (792, 0....  4.474700e-104   
Radial Displacement R'2.5 (mm), Point: (792, -0... -8.621180e-105   
Radial Displacement R'3.5 (mm), Point: (792, -0... -1.101437e-103   
Radial Displacement R'4.5 (mm), Point: (792, 0....  2.933839e-104   
Radial Displacement R1 (mm), Point: (1192, ro, 0)   7.901378e-131   
Radial Displacement R2 (mm), Point: (1192, 0, ro)   1.686566e-132   
Radial Displacement R3 (mm), Point: (1192, -ro, 0)  2.244902e-131   
Radial Displacement R4 (mm), Point: (1192, 0, -ro)  5.775872e-131   
Radial Displacement R1.5 (mm), Point: (1192, 0....  9.096950e-132   
Radial Displacement R2.5 (mm), Point: (1192, -0...  7.989700e-133   
Radial Displacement R3.5 (mm), Point: (1192, -0...  2.518346e-131   
Radial Displacement R4.5 (mm), Point: (1192, 0.... -3.122601e-131   

Time (ms)                                                   0.005  \
Radial Displacement A1 (mm), Point: (21, ro, 0)     -8.245071e-06   
Radial Displacement A2 (mm), Point: (21, 0, ro)     -8.261152e-06   
Radial Displacement A3 (mm), Point: (21, -ro, 0)    -8.244027e-06   
Radial Displacement A4 (mm), Point: (21, 0, -ro)    -8.245081e-06   
Radial Displacement A1.5 (mm), Point: (21, 0.5*...  -8.075622e-06   
Radial Displacement A2.5 (mm), Point: (21, -0.5...  -8.143239e-06   
Radial Displacement A3.5 (mm), Point: (21, -0.5...  -8.082812e-06   
Radial Displacement A4.5 (mm), Point: (21, 0.5*...  -8.088521e-06   
Radial Displacement A'1 (mm), Point: (420, ro, 0)    2.741712e-43   
Radial Displacement A'2 (mm), Point: (420, 0, ro)    1.923536e-45   
Radial Displacement A'3 (mm), Point: (420, -ro, 0)   2.189367e-43   
Radial Displacement A'4 (mm), Point: (420, 0, -ro)   1.960576e-43   
Radial Displacement A'1.5 (mm), Point: (420, 0....   5.221474e-46   
Radial Displacement A'2.5 (mm), Point: (420, -0...  -2.107940e-45   
Radial Displacement A'3.5 (mm), Point: (420, -0...   5.178806e-46   
Radial Displacement A'4.5 (mm), Point: (420, 0....   8.847126e-46   
Radial Displacement R'1 (mm), Point: (792, ro, 0)    1.229687e-77   
Radial Displacement R'2 (mm), Point: (792, 0, ro)    6.135952e-79   
Radial Displacement R'3 (mm), Point: (792, -ro, 0)   1.325349e-77   
Radial Displacement R'4 (mm), Point: (792, 0, -ro)   1.306123e-77   
Radial Displacement R'1.5 (mm), Point: (792, 0....   9.881383e-79   
Radial Displacement R'2.5 (mm), Point: (792, -0...   3.269180e-79   
Radial Displacement R'3.5 (mm), Point: (792, -0...  -1.745170e-78   
Radial Displacement R'4.5 (mm), Point: (792, 0....  -6.882651e-79 

In [169]:
interval = 0.01

Algorithm:
Take the time series, and loop over every value. Mod the value by the interval. If the solution is not zero, erase it and its response column from the dataframe. Save the resulting dataframe as a csv file. 

In [170]:
t = [];
for i in data.columns:
    t.append(i)

In [171]:
time_samples = np.arange(0,data.columns[-1]+interval,interval)
time_samples = np.round(time_samples,2)
strict_data = data

In [172]:
for i in strict_data.columns:
    if i in time_samples:
        continue
    else:
        strict_data = strict_data.drop(i,1)
        
strict_data

C:\Users\MEH-3\AppData\Local\Temp/ipykernel_12264/1020963553.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  strict_data = strict_data.drop(i,1)


Time (ms)                                                    0.00  \
Radial Displacement A1 (mm), Point: (21, ro, 0)     -1.874919e-49   
Radial Displacement A2 (mm), Point: (21, 0, ro)     -1.878951e-49   
Radial Displacement A3 (mm), Point: (21, -ro, 0)    -1.874838e-49   
Radial Displacement A4 (mm), Point: (21, 0, -ro)    -1.874939e-49   
Radial Displacement A1.5 (mm), Point: (21, 0.5*...  -1.847403e-49   
Radial Displacement A2.5 (mm), Point: (21, -0.5...  -1.846380e-49   
Radial Displacement A3.5 (mm), Point: (21, -0.5...  -1.849960e-49   
Radial Displacement A4.5 (mm), Point: (21, 0.5*...  -1.850419e-49   
Radial Displacement A'1 (mm), Point: (420, ro, 0)    3.665651e-78   
Radial Displacement A'2 (mm), Point: (420, 0, ro)    3.119683e-79   
Radial Displacement A'3 (mm), Point: (420, -ro, 0)   3.800742e-78   
Radial Displacement A'4 (mm), Point: (420, 0, -ro)   3.273797e-78   
Radial Displacement A'1.5 (mm), Point: (420, 0....   1.534367e-79   
Radial Displacement A'2.5 (mm), Point: (420, -0...  -1.346842e-79   
Radial Displacement A'3.5 (mm), Point: (420, -0...  -2.857131e-79   
Radial Displacement A'4.5 (mm), Point: (420, 0....   3.872548e-79   
Radial Displacement R'1 (mm), Point: (792, ro, 0)   2.624637e-103   
Radial Displacement R'2 (mm), Point: (792, 0, ro)   2.471391e-104   
Radial Displacement R'3 (mm), Point: (792, -ro, 0)  2.763374e-103   
Radial Displacement R'4 (mm), Point: (792, 0, -ro)  4.284595e-103   
Radial Displacement R'1.5 (mm), Point: (792, 0....  4.474700e-104   
Radial Displacement R'2.5 (mm), Point: (792, -0... -8.621180e-105   
Radial Displacement R'3.5 (mm), Point: (792, -0... -1.101437e-103   
Radial Displacement R'4.5 (mm), Point: (792, 0....  2.933839e-104   
Radial Displacement R1 (mm), Point: (1192, ro, 0)   7.901378e-131   
Radial Displacement R2 (mm), Point: (1192, 0, ro)   1.686566e-132   
Radial Displacement R3 (mm), Point: (1192, -ro, 0)  2.244902e-131   
Radial Displacement R4 (mm), Point: (1192, 0, -ro)  5.775872e-131   
Radial Displacement R1.5 (mm), Point: (1192, 0....  9.096950e-132   
Radial Displacement R2.5 (mm), Point: (1192, -0...  7.989700e-133   
Radial Displacement R3.5 (mm), Point: (1192, -0...  2.518346e-131   
Radial Displacement R4.5 (mm), Point: (1192, 0.... -3.122601e-131   

Time (ms)                                                    0.01  \
Radial Displacement A1 (mm), Point: (21, ro, 0)     -1.592727e-04   
Radial Displacement A2 (mm), Point: (21, 0, ro)     -1.595722e-04   
Radial Displacement A3 (mm), Point: (21, -ro, 0)    -1.592525e-04   
Radial Displacement A4 (mm), Point: (21, 0, -ro)    -1.592727e-04   
Radial Displacement A1.5 (mm), Point: (21, 0.5*...  -1.559870e-04   
Radial Displacement A2.5 (mm), Point: (21, -0.5...  -1.573651e-04   
Radial Displacement A3.5 (mm), Point: (21, -0.5...  -1.561174e-04   
Radial Displacement A4.5 (mm), Point: (21, 0.5*...  -1.562280e-04   
Radial Displacement A'1 (mm), Point: (420, ro, 0)   -8.697846e-42   
Radial Displacement A'2 (mm), Point: (420, 0, ro)   -6.715934e-44   
Radial Displacement A'3 (mm), Point: (420, -ro, 0)  -7.051744e-42   
Radial Displacement A'4 (mm), Point: (420, 0, -ro)  -6.260181e-42   
Radial Displacement A'1.5 (mm), Point: (420, 0....  -1.784380e-44   
Radial Displacement A'2.5 (mm), Point: (420, -0...   5.977349e-44   
Radial Displacement A'3.5 (mm), Point: (420, -0...   5.232210e-45   
Radial Displacement A'4.5 (mm), Point: (420, 0....  -3.724955e-44   
Radial Displacement R'1 (mm), Point: (792, ro, 0)   -1.007121e-75   
Radial Displacement R'2 (mm), Point: (792, 0, ro)   -4.700127e-77   
Radial Displacement R'3 (mm), Point: (792, -ro, 0)  -1.026759e-75   
Radial Displacement R'4 (mm), Point: (792, 0, -ro)  -1.053716e-75   
Radial Displacement R'1.5 (mm), Point: (792, 0....  -7.884181e-77   
Radial Displacement R'2.5 (mm), Point: (792, -0...  -2.442734e-77   
Radial Displacement R'3.5 (mm), Point: (792, -0...   1.243063e-76   
Radial Displacement R'4.5 (mm), Point: (792, 0....   5.091793e-77 

In [173]:
strict_data.to_csv('I:\\My Drive\\USGW Research\\COMSOL Models\\Pipe 4\\Pipe 4\\d33 Displacements\\Viscoelastic\\Inside Circumference\\Short Axis Length\\10 mm Thickness\\30 deg Orientation\\' + sheetname + '.csv')